In [1]:
import pandas as pd

file_name = 'c_Korean Readability Assessment.xlsx' 
df = pd.read_excel(file_name, engine='openpyxl', index_col=None)  # 1579
df = df.dropna()  # 192개의 NaN 제거, 총 1387개
df = df.drop_duplicates(['file_contentCleansed'])  # Nan 제거 후 중복된 98개 제거, 총 1289개
df.head(2) 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,file_url,file_content,file_length,file_contentCleansed,file_contentCleansed_length
0,0,0,217,https://www.ksa.or.kr/krca/ksi/1/217/download.do,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,91401,2003년에는 이러한 의지를 전 임직원이 공유하기 위한‘ 로벌 환경경 선포식’을 갖...,171
2,2,2,219,https://www.ksa.or.kr/krca/ksi/1/219/download.do,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,8303,"세계경제의성장과기업활동의글로벌화로전세계인류는풍요로운삶을 영위할수 있었으나, 다른 한...",27


In [2]:
# 전처리
import re

def clean_text(input_setence):
    corpus = ''
    special = re.compile(r'[^가-힣+\.\" "]')  # 한글과 온점을 제외한 글자 제거
    content = special.sub('', input_setence)  
    content = re.sub(r'\s+', ' ', content)  # extra space 제거
    corpus += content
    return corpus


In [3]:
# test data
data = df.iloc[0,6]
data

'2003년에는 이러한 의지를 전 임직원이 공유하기 위한‘ 로벌 환경경 선포식’을 갖고 환경부문에서도 2010년 로벌 톱5에 진입하겠다는 강력한 의지를 천명한바 있으며, 국내 자동차업계 최초로 환경기술분야의 핵심 연구를 독자적이고 체계적으로 수행할 환경기술연구소의 설립을 통해 환경친화적 자동차의 개발에도 최선을다하고 있습니다. 또한 생산, A/S 등 각 부문에서 ISO 14001과 같은 국제적인 환경경 시스템 인증을 획득, 환경을 우선시한 청정 생산 활동과 제품의 공급, 서비스를펼쳐가고 있습니다 대표이사인사말2004 Environmental & Social Report  9사회적, 경제적 이해관계자로서의 책임을 다해야 하는 기업의 사회적 역할 실현을위해서는 윤리경 을 도입, 투명한 경 활동으로 이익을 창출하고 이를 통하여 지속적인 고용창출을 이루어내고 있습니다. 기아자동차 임직원은 미래에 형성될 첨단 자동차 사회에 더욱 안전하고 가치 있는 제품을 공급하는 것이 인간과 환경을 위한 가장 중요한 사명이라고 생각하고 있습니다 지속가능한 발전은 현재와 미래를 위한 기아자동차의 약속입니다 기아자동차는 풍요롭고 지속가능한 사회의 구현과 기업의 사회적 책임을 다하기 위한 노력을 앞으로도 계속해 나갈 것을 약속드립니다.(KCI), 유럽지역 판매를 위한독일의 Kia Motors Deutschland GmbH(KMD) 및 Kia MotorsEurope GmbH(KME) 등이 있습니다. 윤리경 및투명경 선언환경방침을개정한환경선언및환경방침공포ECO GT-5달성을위한환경경 선포식자율환경관리협약체결(광주공장) 환경기술연구소기공식소하리, 화성, 광주및A/S사업부 ISO 14001인증광주공장환경친화기업지정환경 연혁12 2004 Environmental & Social Report2004 Environmental & Social Report  13지속가능한비전과전략자동차는인간에의해탄생되어인간의요구에맞게발달되어왔으며, 또한새로운인간의요구인자연이라는요구에맞추어인간과자연이조화하는방향으로발전될것입

In [4]:
# fog_index
import re
from pororo import Pororo

tk = Pororo(task="tokenization", lang="ko", model="bpe32k.ko")  # pororo tokenizer
num_syllable = int(input())  # 음절 입력

def ra_fogIndex(input_sentence:str, n=num_syllable, tokenizer=tk):
    cleaned_input_sentence = clean_text(input_sentence)             # 전처리
    input_sentence_length = len(cleaned_input_sentence.split('.'))  # 문장 수
    input_sentence_tokenzied = tokenizer(cleaned_input_sentence)    # 토큰화

    num_letter = 0  # 글자 수
    long_words = 0  # n음절 이상인 단어 수
    total_word = 0  # 단어 수

    for word in input_sentence_tokenzied:   # 단어 수는 빈칸(" ")으로 구분
        word = re.sub(r'▁', '', word)       # _ 제거
        word = re.sub(r'\.', '', word)      # . 제거
        
        if word != '':
            num_letter += len(word)
            if len(word) == n:
                long_words += 1
            if len(word) >= 2:
                total_word += 1

    ASL = total_word / input_sentence_length  # 평균 문장 길이
    fog_index = (ASL + (long_words / total_word) * 100) * 0.4
    
    return fog_index

print(ra_fogIndex(data))

10.272772097425682


In [5]:
# flesch
import re
from pororo import Pororo

tk = Pororo(task="tokenization", lang="ko", model="bpe32k.ko")  # pororo tokenizer

def ra_flesch(input_sentence:str, n=num_syllable, tokenizer=tk):
    cleaned_input_sentence = clean_text(input_sentence)             # 전처리
    input_sentence_length = len(cleaned_input_sentence.split('.'))  # 문장 수
    input_sentence_tokenzied = tokenizer(cleaned_input_sentence)    # 토큰화

    num_letter = 0  # 글자 수
    total_word = 0  # 단어 수

    for word in input_sentence_tokenzied:  
        word = re.sub(r'▁', '', word)       # _ 제거
        word = re.sub(r'\.', '', word)      # . 제거
        
        if word != '':
            num_letter += len(word)
            if len(word) >= 2:
                total_word += 1

    AWL = num_letter / total_word             # 평균 단어 길이
    ASL = total_word / input_sentence_length  # 평균 문장 길이
    flesch = 206.835 - 84.6 * AWL - 1.015 * ASL    
     
    return flesch

print(ra_flesch(data))

-63.175060285717414


In [6]:
# length
import re
import math
from pororo import Pororo

tk = Pororo(task="tokenization", lang="ko", model="bpe32k.ko")  # pororo tokenizer

def ra_length(input_sentence:str, n=num_syllable, tokenizer=tk):
    cleaned_input_sentence = clean_text(input_sentence)             # 전처리
    input_sentence_length = len(cleaned_input_sentence.split('.'))  # 문장 수
    input_sentence_tokenzied = tokenizer(cleaned_input_sentence)    # 토큰화

    total_word = 0  # 단어 수

    for word in input_sentence_tokenzied:  
        
        word = re.sub(r'▁', '', word)       # _ 제거
        word = re.sub(r'\.', '', word)      # . 제거
        
        if word != '':
            if len(word) >= 2:
                total_word += 1

    length = math.log(total_word)     # 문서의 길이 측정
    return length

print(ra_length(data))

8.349011198176003
